In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn import metrics
from functions import important_features

In [2]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('data') 

# Actions that we try to detect
actions = np.array(["idle","my", "name", "hello","I","live at"]) #change according to ur required gestures!!


# thirty videos worth of data
no_sequences = 30

# Videos are going to be 10 frames in length
sequence_length = 11

# Folder start
start_folder = 30

In [3]:
label_map = {label:num for num, label in enumerate(actions)}

In [5]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            i = 0
            if(frame_num in [0, 5, 11]):
                imp = np.load(os.path.join(DATA_PATH, action, str(sequence), "important.npy"))
                res = np.concatenate([res, imp[i]])
                i = i+1
            else:
                res = np.concatenate([res,np.array([0,0,0,0])])
            window.append(res)
                
            
        sequences.append(window)
        labels.append(label_map[action])
        

In [6]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [7]:
X.shape

(180, 11, 146)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.10, random_state=34, stratify=y)

In [9]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(11,146)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(32, return_sequences=False, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [10]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 11, 32)            22912     
                                                                 
 lstm_1 (LSTM)               (None, 11, 64)            24832     
                                                                 
 lstm_2 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                                                 
Total params: 61,414
Trainable params: 61,414
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X_train, Y_train, epochs=25)

Epoch 1/25
6/6 [==============================] - 5s 18ms/step - loss: 1.7155 - categorical_accuracy: 0.3580
Epoch 2/25
6/6 [==============================] - 0s 17ms/step - loss: 1.3904 - categorical_accuracy: 0.5062
Epoch 3/25
6/6 [==============================] - 0s 16ms/step - loss: 1.0201 - categorical_accuracy: 0.6605
Epoch 4/25
6/6 [==============================] - 0s 16ms/step - loss: 0.6022 - categorical_accuracy: 0.7716
Epoch 5/25
6/6 [==============================] - 0s 16ms/step - loss: 0.9399 - categorical_accuracy: 0.7222
Epoch 6/25
6/6 [==============================] - 0s 18ms/step - loss: 0.7815 - categorical_accuracy: 0.6605
Epoch 7/25
6/6 [==============================] - 0s 17ms/step - loss: 0.6664 - categorical_accuracy: 0.8333
Epoch 8/25
6/6 [==============================] - 0s 17ms/step - loss: 0.5981 - categorical_accuracy: 0.7716
Epoch 9/25
6/6 [==============================] - 0s 17ms/step - loss: 0.4684 - categorical_accuracy: 0.8827
Epoch 10/25
6/6 [==

In [12]:
model.save('model-words-2')

INFO:tensorflow:Assets written to: model-words-2\assets


INFO:tensorflow:Assets written to: model-words-2\assets


In [13]:
predictions = np.argmax(model.predict(X_test), axis=1)

1/1 [==============================] - 1s 587ms/step


In [14]:
test_labels = np.argmax(Y_test, axis=1)

In [15]:
accuracy = metrics.accuracy_score(test_labels, predictions)

In [16]:
accuracy

1.0